### Training QSVM model on NSL-KDD dataset with Qiskit

In [1]:
# Importing the necessary modules
from qiskit import Aer
from qiskit.ml.datasets import ad_hoc_data
from qiskit.circuit.library import ZZFeatureMap
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.components.multiclass_extensions import AllPairs


ModuleNotFoundError: No module named 'qiskit.ml'

Load NSL-KDD data

In [ ]:
feature_dim = 10  # Ajuster la dimension en fonction de tes besoins
training_size = 20
test_size = 10
n = 2  # Nombre de qubits
sample_Total, training_input, test_input, class_labels = ad_hoc_data(
    training_size, test_size, n, gap=0.3
)

Preprocess NSL-KDD data

Creation of the quantum circuit

In [ ]:
# Création du circuit quantique avec ZZFeatureMap
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=2)


Configuring the Quantum Instance

In [ ]:
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=10598, seed_transpiler=10598)


Creating the QSVM model

In [ ]:
qsvm = QSVM(feature_map, training_input, test_input, total_array, multiclass_extension=AllPairs())


Model training

In [ ]:
result = qsvm.run(quantum_instance)


Model evaluation

In [ ]:
print(f'Testing success ratio: {result["testing_accuracy"]}')
